In [3]:
import pandas as pd
import joblib 
from sentence_transformers import SentenceTransformer, util
from rank_bm25 import BM25Okapi
import string
import nltk
from nltk.corpus import stopwords
import torch
import os
import pickle
from sklearn.linear_model import LogisticRegression # Kita butuh ini untuk type-checking

# --- KONFIGURASI PATH ---
# Notebook ada di folder scripts/, jadi naik 1 level ke root
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '..'))

MODEL_DIR = os.path.join(ROOT_DIR, 'models')
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'processed')

# FILE-FILE YANG DIBUTUHKAN
CSV_PATH = os.path.join(DATA_DIR, 'dataset_training_FULL_COMPLETE.csv')
EMB_FILE = os.path.join(MODEL_DIR, 'corpus_embeddings.pt')
LR_PATH = os.path.join(MODEL_DIR, 'logisticregression (3).pkl') # Model Teman Anda

# URUTAN FITUR WAJIB (Sesuai isi file .pkl teman Anda)
LR_FEATURES = ['sbert_sim', 'overlap_score', 'jaccard_score', 'bm25_score'] 

print("✅ Path berhasil diset.")
print(f"📂 Root Dir: {ROOT_DIR}")
print(f"📂 Data Dir: {DATA_DIR}")
print(f"\n🔍 Checking files:")
print(f"   CSV exists: {os.path.exists(CSV_PATH)}")
print(f"   - Path: {CSV_PATH}")
print(f"   Embedding exists: {os.path.exists(EMB_FILE)}")
print(f"   LogReg exists: {os.path.exists(LR_PATH)}")

✅ Path berhasil diset.
📂 Root Dir: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir
📂 Data Dir: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\data\processed

🔍 Checking files:
   CSV exists: True
   - Path: c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\data\processed\dataset_training_FULL_COMPLETE.csv
   Embedding exists: True
   LogReg exists: True


In [4]:
print("⚙️ Memuat Dataset Lengkap...")

if not os.path.exists(CSV_PATH):
    raise FileNotFoundError(f"File {CSV_PATH} tidak ditemukan!")

# 1. Baca CSV
df_full = pd.read_csv(CSV_PATH)
df_full.columns = df_full.columns.str.strip().str.lower()

# 2. Ambil Tafsir Unik (Corpus Pencarian)
df_index = df_full.drop_duplicates(subset=['text']).copy()
unique_tafsirs = df_index['text'].astype(str).tolist()

# 3. Buat Kamus Metadata
metadata_map = {}
for _, row in df_index.iterrows():
    key = str(row['text']).strip()
    metadata_map[key] = {
        'lokasi': row.get('ayat_asal', '?'),
        'arabic': row.get('arabic', ''),
        'trans': row.get('translation', '')
    }

print(f"✅ Data Termuat: {len(unique_tafsirs)} dokumen tafsir unik.")

⚙️ Memuat Dataset Lengkap...
✅ Data Termuat: 6031 dokumen tafsir unik.


In [5]:
print("⚙️ Memuat Embedding & SBERT...")

# 1. Load Embedding Lama (.pt)
if os.path.exists(EMB_FILE):
    corpus_embeddings = torch.load(EMB_FILE, map_location='cpu')
    print(f"✅ Embedding Lama Berhasil Diload.")
else:
    raise FileNotFoundError("File corpus_embeddings.pt hilang!")

# 2. Load SBERT (Hanya untuk encode query)
SBERT_FOLDER = os.path.join(MODEL_DIR, 'sbert_finetuned_quran')
try:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu')
    print("✅ SBERT Loaded.")
except:
    sbert_model = SentenceTransformer(SBERT_FOLDER, device='cpu')

⚙️ Memuat Embedding & SBERT...
✅ Embedding Lama Berhasil Diload.


The tokenizer you are loading from 'c:\Kuliah ITS Farhan\Semester 3\A_Final_Project\ML_DM\fp-quran-ir-query-tafsir\models\sbert_finetuned_quran' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


✅ SBERT Loaded.


In [6]:
print("⚙️ Memuat Model Regresi Logistik...")

if not os.path.exists(LR_PATH):
    raise FileNotFoundError(f"Model {LR_PATH} tidak ditemukan.")

# 1. Load Model
try:
    # LR biasanya disimpan dengan joblib, atau kadang dengan pickle biasa
    try:
        lr_model = joblib.load(LR_PATH)
    except:
        with open(LR_PATH, 'rb') as f:
            lr_model = pickle.load(f)
            
    print("✅ Model LR berhasil dimuat.")
    
    # Cek apakah objek yang diload benar-benar model LR
    if not isinstance(lr_model, LogisticRegression):
         print(f"⚠️ Objek yang diload bukan LogisticRegression, melainkan {type(lr_model)}. Hasil prediksi mungkin error.")
         
except Exception as e:
    print(f"❌ Gagal memuat model LR: {e}")
    raise SystemExit("Model LR gagal dimuat.")

⚙️ Memuat Model Regresi Logistik...
✅ Model LR berhasil dimuat.


c:\Users\Farhan\AppData\Local\Programs\Python\Python314\Lib\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.2.2 when using version 1.7.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [7]:
print("⚙️ Menyiapkan BM25...")

try:
    nltk.download('stopwords', quiet=True)
    stop_words = set(stopwords.words('indonesian'))
except:
    stop_words = set(['yang', 'dan', 'di'])

def clean_tokens(text):
    text = str(text).lower().translate(str.maketrans('', '', string.punctuation))
    return [w for w in text.split() if w not in stop_words]

# Build Index BM25
corpus_tokens = [clean_tokens(t) for t in unique_tafsirs]
bm25 = BM25Okapi(corpus_tokens)

# Pastikan fungsi clean_tokens dapat diakses di cell lain
clean_func = clean_tokens 

print("✅ BM25 Siap.")

⚙️ Menyiapkan BM25...
✅ BM25 Siap.


In [8]:
def cari_regresi_logistik(query_text, threshold=0.5): # Threshold default 0.5 untuk LR
    print(f"\n🔍 QUERY: '{query_text}'")
    print(f"⚙️ Threshold yang digunakan: {threshold:.4f}")
    print("=" * 60)
    
    # 1. Retrieval (SBERT)
    query_vec = sbert_model.encode(query_text, convert_to_tensor=True)
    hits = util.semantic_search(query_vec, corpus_embeddings, top_k=50)[0]
    
    candidates = []
    q_toks = clean_func(query_text)
    
    # 2. Hitung Fitur
    for hit in hits:
        idx = hit['corpus_id']
        txt = unique_tafsirs[idx]
        
        # Hitung Fitur
        t_toks = clean_func(txt)
        sq, st = set(q_toks), set(t_toks)
        ov = len(sq & st) / len(sq) if sq else 0
        jac = len(sq & st) / (len(sq | st) + 1e-9)
        bm25_s = bm25.get_batch_scores(q_toks, [idx])[0]
        
        candidates.append({
            'text_key': txt,
            'sbert_sim': hit['score'],
            'bm25_score': bm25_s,
            'overlap_score': ov,
            'jaccard_score': jac
        })
    
    # 3. Prediksi dengan Regresi Logistik
    df_cand = pd.DataFrame(candidates)
    
    # SUSUN KOLOM SESUAI URUTAN MODEL TEMAN ANDA (WAJIB!)
    X_pred = df_cand[LR_FEATURES]
    
    # Regresi Logistik predict_proba (ambil probabilitas kelas 1)
    try:
        # Paling umum, ambil probabilitas kelas 1
        scores = lr_model.predict_proba(X_pred)[:, 1]
    except AttributeError:
        # Jika model dilatih tanpa predict_proba (jarang untuk LR)
        scores = lr_model.predict(X_pred)
    
    df_cand['final_score'] = scores
    
    # 4. Filter & Tampilkan
    results = df_cand[df_cand['final_score'] > threshold].sort_values('final_score', ascending=False)
    
    if len(results) == 0:
        print("❌ Tidak ada hasil relevan (Coba turunkan threshold).")
    else:
        print(f"✅ Top 3 Hasil (Engine: Regresi Logistik):\n")
        
        for i, row in enumerate(results.head(3).iterrows()):
            _, data = row
            key = str(data['text_key']).strip()
            score = data['final_score']
            info = metadata_map.get(key)
            
            print(f"🏅 [RANK {i+1}] Skor: {score:.4f}")
            if info:
                print(f"📍 {info['lokasi']}")
                print(f"🇸🇦 {info['arabic'][:60]}...")
                print(f"🇮🇩 {info['trans'][:100]}...")
            print(f"💬 TAFSIR: {key[:100]}...")
            print("-" * 50)

In [9]:
cari_regresi_logistik("Kisah sapi betina bani israil")
cari_regresi_logistik("Hukum warisan bagi perempuan")


🔍 QUERY: 'Kisah sapi betina bani israil'
⚙️ Threshold yang digunakan: 0.5000
❌ Tidak ada hasil relevan (Coba turunkan threshold).

🔍 QUERY: 'Hukum warisan bagi perempuan'
⚙️ Threshold yang digunakan: 0.5000
✅ Top 3 Hasil (Engine: Regresi Logistik):

🏅 [RANK 1] Skor: 0.7355
📍 QS. An-Nisā'  : Ayat 11
🇸🇦 يُوْصِيْكُمُ اللّٰهُ فِيْٓ اَوْلَادِكُمْ لِلذَّكَرِ مِثْلُ ح...
🇮🇩 Allah mensyariatkan (mewajibkan) kepadamu tentang (pembagian warisan untuk) anak-anakmu, (yaitu) bag...
💬 TAFSIR: Setelah ayat sebelumnya menjelaskan dampak orang yang mengabaikan hak orang lain, ayat ini menjelask...
--------------------------------------------------
🏅 [RANK 2] Skor: 0.6486
📍 QS. An-Nisā'  : Ayat 127
🇸🇦 وَيَسْتَفْتُوْنَكَ فِى النِّسَاۤءِۗ  قُلِ اللّٰهُ يُفْتِيْكُ...
🇮🇩 Mereka meminta fatwa kepada engkau (Nabi Muhammad) tentang perempuan. Katakanlah, “Allah memberi fat...
💬 TAFSIR: Dan mereka meminta fatwa, yaitu penjelasan hukum, kepadamu, wahai Muhammad, tentang berbagai persoal...
---------------------